# Investment Chatbot in Python (using NLTK)


### Import necessary libraries


In [1]:
import io
import random
import string # to process standard python strings
import warnings
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

### Installing NLTK Packages


In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('popular', quiet=True) # for downloading packages
# nltk.download('punkt') # first-time use only
# nltk.download('wordnet') # first-time use only

True

### Reading in the corpus

In [5]:
with open('chatbot.txt','r', encoding='utf8', errors ='ignore') as fin:
    raw = fin.read()
    raw = raw.lower() # converts to lowercase

### Tokenisation

In [6]:
# converts to list of sentences
sent_tokens = nltk.sent_tokenize(raw)
print(sent_tokens)

['what is your name?', 'my name is robo and i will help you with your investment queries today.', 'invest\nthere are many options to invest:\n1. regional or investments banks\n2. stocks\nin which section would you like to invest?', 'money\nthere are many options to invest:\n1. regional or investments banks\n2. stocks \\n\nin which section would you like to invest?', 'where to put\nthere are many options to invest:\n1. regional or investments banks\n2. stocks \\n\nin which section would you like to invest?', 'loans\nhousing,personal,educational.i recommend you to visit sbi banks for this.', 'investments banks\nwell there are many such as: \nubs\nbarclays\ndeutsche bank\nhsbc\nwells fargo\n\nregional banks\nthere are many: \nsbi\nidbi\nbob\nkotak\niob\nicici\nchoose any one to know more.', 'sbi\nsbi offers loan at 10% interest.', 'iob\niob offers loan at 9.50% interest.', 'idbi\nkotak offers loan at 07% interest.', 'bob\nbob offers loan at 11% interest.', 'kotak\nkotak offers loan at 08%

In [7]:
# converts to list of words
word_tokens = nltk.word_tokenize(raw)
print(word_tokens)

['what', 'is', 'your', 'name', '?', 'my', 'name', 'is', 'robo', 'and', 'i', 'will', 'help', 'you', 'with', 'your', 'investment', 'queries', 'today', '.', 'invest', 'there', 'are', 'many', 'options', 'to', 'invest', ':', '1.', 'regional', 'or', 'investments', 'banks', '2.', 'stocks', 'in', 'which', 'section', 'would', 'you', 'like', 'to', 'invest', '?', 'money', 'there', 'are', 'many', 'options', 'to', 'invest', ':', '1.', 'regional', 'or', 'investments', 'banks', '2.', 'stocks', '\\n', 'in', 'which', 'section', 'would', 'you', 'like', 'to', 'invest', '?', 'where', 'to', 'put', 'there', 'are', 'many', 'options', 'to', 'invest', ':', '1.', 'regional', 'or', 'investments', 'banks', '2.', 'stocks', '\\n', 'in', 'which', 'section', 'would', 'you', 'like', 'to', 'invest', '?', 'loans', 'housing', ',', 'personal', ',', 'educational.i', 'recommend', 'you', 'to', 'visit', 'sbi', 'banks', 'for', 'this', '.', 'investments', 'banks', 'well', 'there', 'are', 'many', 'such', 'as', ':', 'ubs', 'barcl

### Preprocessing

Function called LemTokens is defined, which will take the tokens as input and return normalized tokens.

In [8]:
lemmer = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))


### Keyword Matching

Next, we define a function for a greeting by the bot.
i.e if a user’s input is a greeting, the bot shall return a greeting response.

In [9]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey","Namaste")
GREETING_RESPONSES = ["hi, how can i help you?", "hey", "hi there", "hello", "I am glad! You are talking to me"]

In [10]:
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

To generate a response from  chat bot for input questions, the concept of <b>document similarity</b> will be used. 

A function response which searches the user’s utterance for one or more known keywords and returns one of several possible responses. 

If it doesn’t find the input matching any of the keywords, it returns a response:” I am sorry! I don’t understand you”

In [13]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

Lines that chat bot will say while <b>starting and ending</b> a conversation depending upon user’s input.

In [14]:
flag=True
print("ROBO: My name is Robo. I will answer your queries about Investments. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                res = response(user_response)
                nlines = res.count('\n')
                if nlines > 0:
                    res = res.split("\n",1)[1]
                print(res)
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")

ROBO: My name is Robo. I will answer your queries about Investments. If you want to exit, type Bye!
hello
ROBO: hello
loans
ROBO: kotak offers loan at 07% interest.
invest
ROBO: there are many options to invest:
1. regional or investments banks
2. stocks
in which section would you like to invest?
regional banks
ROBO: well there are many such as: 
ubs
barclays
deutsche bank
hsbc
wells fargo

regional banks
there are many: 
sbi
idbi
bob
kotak
iob
icici
choose any one to know more.
sbi
ROBO: sbi offers loan at 10% interest.
icici
ROBO: icici offers a loan at 7.5% intrest.
idbi
ROBO: kotak offers loan at 07% interest.
stocks
ROBO: we have to companies to offer:
zoho
reliance
choose any one to know more.
gdhj
ROBO: I am sorry! I don't understand you
zoho
ROBO: the company zoho has a roi = 15%.
reliance
ROBO: the company reliance has a roi = 14%.
money
ROBO: there are many options to invest:
1. regional or investments banks
2. stocks \n
in which section would you like to invest?
bye
ROBO: By